In [ ]:
#IMPORTS INICIALES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autoencoderManager import AutoencoderManager
from fonts import getCharacterMatrix,getCharacterMap,getFormattedFont
from helpers.configHelper import ConfigHelper
from graph import plotErrorAgainstSteps
import pickle
from os.path import exists


# TP5 - Deep Learning

In [ ]:
#LEEMOS LOS PARAMETROS DEL CONFIG HELPER

configPath="./config/config.json"
configHelper = ConfigHelper(configPath)
(architecture,encoderActivationFunction,latentSpaceActivationFunction,decoderActivationFunction,encoderBeta,latentSpaceBeta,decoderBeta,learningRate,maxEpochs,maxToleranceExponent,randomSeed,font) = configHelper.getProperties()

In [ ]:
#PEDIR LA FONT A UTILIZAR Y ARMAR EL TRAINING SET Y RESULT SET
formattedFont = getFormattedFont(font)
getCharacterMap(formattedFont[2])
trainingSet = getFormattedFont(font,flatten=True)
resultsSet = trainingSet
print(trainingSet)

In [ ]:
# #Seteo de seed
# np.random.seed(randomSeed)
# random.seed(randomSeed)
# neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxEpochs,maxToleranceExponent)
# (epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)

# #autoencoderManager = AutoencoderManager(architecture,activationFunction,learningRate,maxEpochs,errorHelper)
# #(epochs,executionTime,exception) = autoencoderManager.start(trainingSet,resultsSet)

# #Plot error graph
# plotEpochsError(epochs)
# #Print output
# print("FINISH-------------------------------------------------------------------------------------------")
# output = Output(configHelper,epochs[-1].error,epochs[-1].epochNumber,executionTime)
# output.printOutput()

In [ ]:
# ENTRENAMIENTO DE LA RED
PATH = "autoencoderW.pickle"
autoencoderManager = AutoencoderManager(architecture,encoderActivationFunction,latentSpaceActivationFunction,decoderActivationFunction,learningRate,maxEpochs)
if exists(PATH):
    print("INITIALIZING WEIGHTS FOR CONFIGURATION")
    file = open(PATH,'rb')
    wFinal = pickle.load(file)
    file.close()
    autoencoderManager.initilizeWeights(trainingSet,wFinal)
else:
    print("CREATING WEIGHTS FOR CONFIGURATION")
    (wFinal,finalError) = autoencoderManager.start(trainingSet)
    file = open(PATH,'wb')
    pickle.dump(wFinal,file)
    file.close()

print(wFinal)
    


In [ ]:
# LATENT SPACE CONFIGURATION
latentSpaceData = {
    'X' : [],
    'Y' : []
}
for character in trainingSet:
    point = autoencoderManager.getLatentSpaceConfig(character)
    latentSpaceData['X'].append(point[0])
    latentSpaceData['Y'].append(point[1])
data = pd.DataFrame(latentSpaceData)
print(latentSpaceData)
sns.scatterplot(x="X",y="Y",data=data)

In [ ]:
#GRAFICAR ERROR VS STEPS
plotErrorAgainstSteps(autoencoderManager.errors,autoencoderManager.steps)
print(autoencoderManager.steps)

In [ ]:
# PLOT INPUT VS OUTPUT OF FINAL LAYER
figure, axes = plt.subplots( 2,len(formattedFont), sharex=True, figsize=(19,7))
for font in range(0,len(formattedFont)):
    getCharacterMap(formattedFont[font],ax=axes[0,font])
    umbralFormattedCharacter = [x if x>=0.35 else 0 for x in autoencoderManager.propagate(trainingSet[font])]
    getCharacterMap(getCharacterMatrix(np.array(umbralFormattedCharacter)),ax=axes[1,font])